In [ ]:
import numpy as np
import tensorflow as tf
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

2025-06-06 17:18:46.953457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749230326.981805    2126 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749230326.990342    2126 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
X = np.loadtxt("/kaggle/input/coffee-roasting-data-binary/input.csv", delimiter=",")
Y = np.loadtxt("/kaggle/input/coffee-roasting-data-binary/output.csv", delimiter=",")
Y = Y.reshape(-1,1)

In [ ]:
Xt = np.tile(X,(1000,1))
Yt = np.tile(Y,(1000,1))   
print(Xt.shape, Yt.shape) 

(200000, 2) (200000, 1)


In [ ]:
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(2, activation='linear',  name = 'layer1'),
        Dense(2, activation='sigmoid', name = 'layer2'),
        Dense(1, activation='sigmoid', name = 'layer3')
     ]
)

2025-06-06 17:18:51.376643: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                       │ (None, 2)                   │               6 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer2 (Dense)                       │ (None, 2)                   │               6 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer3 (Dense)                       │ (None, 1)                   │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15 (60.00 B)

 Trainable params: 15 (60.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1),
)

In [ ]:
W1 = np.array([
    [-1.0, 5.0],
    [10.0, 7.0]] )
W2 = np.array([
    [-15.0, -1.0],
	[10.0, 7.0]])
W3 = np.array([
    [-12.0],
    [-15.0]])

b1 = np.array([-41.0, 22.0])
b2 = np.array([-28.0, -91.0])
b3 = np.array([31.0])

model.get_layer("layer1").set_weights([W1,b1])
model.get_layer("layer2").set_weights([W2,b2])
model.get_layer("layer3").set_weights([W3,b3])

In [ ]:
model.fit(
    Xt,Yt,  
    epochs = 10,
    batch_size = 25000,
    # steps_per_epoch = 8,
    shuffle = False,
)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.8239  
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.5999 
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8851 
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6330 
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5585
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5345 
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5260 
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5227 
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.5214
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.5209


In [ ]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
W3, b3 = model.get_layer("layer3").get_weights()

print("W1:\n", W1, "\nb1:", b1)
print("\nW2:\n", W2, "\nb2:", b2)
print("\nW3:\n", W3, "\nb3:", b3)

W1:
 [[-1.  5.]
 [10.  7.]] 
b1: [-41.  22.]

W2:
 [[-15.  -1.]
 [ 10.   7.]] 
b2: [-28. -91.]

W3:
 [[-13.748726]
 [-16.74873 ]] 
b3: [29.25127]
